Consumer Financial Protection Bureau

Debt Collection Every Two Years

In [194]:
#import files
import os
import json
import pprint
import pandas as pd
import numpy as np
import requests

In [195]:
#import files

#"https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1
#/?date_received_max=2023-07-05&date_received_min=2020-01-05&field=all&format=json&lens=product&no_aggs=true&product=Debt%20collection&size=216013&sub_lens=sub_product&trend_depth=5&trend_interval=month"
# Complaints 2017-2020

with open("debt_collection 2021-2023.json") as debt_collect_2123:
    debt_collect_2123 = json.load(debt_collect_2123) 

    

In [ ]:
#import by API

In [ ]:
#pprint.pprint(debt_collect_2123[0]['_source'])
#{'company': 'ENCORE CAPITAL GROUP INC.',
# 'company_public_response': None,
# 'company_response': 'Closed with non-monetary relief',
# 'complaint_id': '7089490',
# 'complaint_what_happened': '',
# 'consumer_consent_provided': None,
# 'consumer_disputed': 'N/A',
# 'date_received': '2023-06-08T12:00:00-05:00',
# 'date_sent_to_company': '2023-06-08T12:00:00-05:00',
# 'issue': 'Written notification about debt',
# 'product': 'Debt collection',
# 'state': 'TN',
# 'sub_issue': "Didn't receive enough information to verify debt",
# 'sub_product': 'I do not know',
# 'submitted_via': 'Web',
# 'tags': None,
# 'timely': 'Yes',
# 'zip_code': '38128'}


In [196]:
issue = []
sub_issue =[]
company = []
company_response = []
complaint_long_desc = []
consumer_disputed =[]
state = []
timely = []
zip_code = []

for x in range(len(debt_collect_2123)): 
    company.append(debt_collect_2123[x]['_source']['company'])
    issue.append(debt_collect_2123[x]['_source']['issue'])
    sub_issue.append(debt_collect_2123[x]['_source']['sub_issue'])
    company_response.append(debt_collect_2123[x]['_source']['company_response'])
    consumer_disputed.append(debt_collect_2123[x]['_source']['consumer_disputed'])
    timely.append(debt_collect_2123[x]['_source']['timely'])
    state.append(debt_collect_2123[x]['_source']['state'])
    zip_code.append(debt_collect_2123[x]['_source']['zip_code'])
    
debt_2123_df = pd.DataFrame({'issue': issue, "company":company, "company_response":company_response, "timely" :timely,"state":state, "zip_code": zip_code })

print(debt_2123_df.columns)
debt_2123_df= debt_2123_df.set_index('timely')
debt_2123_df= debt_2123_df.loc[['No']].set_index('issue')
debt_2123_df

Index(['issue', 'company', 'company_response', 'timely', 'state', 'zip_code'], dtype='object')


,company,company_response,state,zip_code
issue,,,,
Communication tactics,"Vital Solutions, Inc.",Closed with explanation,PA,184XX
Attempts to collect debt not owed,"The Law Offices of Kenosian & Miele, LLP",Closed with explanation,CA,90066
Written notification about debt,"HOVG, LLC",Closed with explanation,FL,33176
Took or threatened to take negative or legal action,"P&B Capital Group, LLC",Closed with explanation,FL,32828
Took or threatened to take negative or legal action,Bridgecrest Acceptance Corporation,Closed with explanation,GA,30260
...,...,...,...,...
Attempts to collect debt not owed,"The Thomas Agency, Inc.",Untimely response,GA,30046
Attempts to collect debt not owed,"The Thomas Agency, Inc.",Untimely response,NH,03465
Attempts to collect debt not owed,"Best Account Receivables Management Solutions,...",Untimely response,CA,93305


In [236]:
o2123_df=debt_2123_df.groupby(['issue', 'company_response','company','state','zip_code']).count().reset_index() 

#o2123_df=o2123_df.drop(o2123_df[o2123_df.apply(lambda x: x.map(o2123_df.stack().value_counts()) == 2)].index)
#condition = o2123_df.company.value_counts() <= 2
#o2123_df = o2123_df.drop(o2123_df[o2123_df.company.value_counts() <= 2].index)

# Drop rows based on the condition using an if statement
indices_to_drop = []
for i in range(len(o2123_df)):
    value = o2123_df.at[i, 'company']
    count = (o2123_df['company'] == value).sum()
    if count <= 100:  # Set the condition directly (count <= 1)
        indices_to_drop.append(i)
# Drop the rows from the DataFrame using the indices
o2123_df.drop(indices_to_drop, inplace=True)

# Reset the index if needed
o2123_df.reset_index(drop=True, inplace=True)
o2123_df['company'].value_counts()
#o2123_df=o2123_df.groupby(['company','issue', 'company_response','state','zip_code']).count()


CCS Financial Services, Inc.                 268
Capital Accounts, LLC                        230
AmerAssist A/R Solutions, Inc.               190
BANK OF AMERICA, NATIONAL ASSOCIATION        156
HCFS Health Care Financial Services, Inc.    152
Commonwealth Financial Systems, Inc.         120
WAKEFIELD & ASSOCIATES, INC.                 119
ResidentCollect, Inc.                        106
Name: company, dtype: int64

In [ ]:
#debt_2123_df = debt_2123_df.groupby(debt_2123_df.company).agg({
#    'issue': lambda x: ','.join(x.unique())
#    'issue': ['count', 'mean'],
#})

o2123_df.loc[o2123_df['company'].value_counts() != 1]

In [ ]:
debt_2123_df.head()